# Случайные леса
__Суммарное количество баллов: 10__

__Решение отправлять на `ml.course.practice@gmail.com`__

__Тема письма: `[HSE][ML][MS][HW09] <ФИ>`, где вместо `<ФИ>` указаны фамилия и имя__

В этом задании вам предстоит реализовать ансамбль деревьев решений, известный как случайный лес, применить его к публичным данным пользователей социальной сети Вконтакте, и сравнить его эффективность с ансамблем, предоставляемым библиотекой CatBoost.

В результате мы сможем определить, какие подписки пользователей больше всего влияют на определение возраста и пола человека. 

In [1]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas
import random
import matplotlib.pyplot as plt
import matplotlib
import copy
from catboost import CatBoostClassifier

### Задание 1 (2 балла)
Random Forest состоит из деревьев решений. Каждое такое дерево строится на одной из выборок, полученных при помощи bagging. Элементы, которые не вошли в новую обучающую выборку, образуют out-of-bag выборку. Кроме того, в каждом узле дерева мы случайным образом выбираем набор из `max_features` и ищем признак для предиката разбиения только в этом наборе.

Сегодня мы будем работать только с бинарными признаками, поэтому нет необходимости выбирать значение признака для разбиения.

#### Методы
`predict(X)` - возвращает предсказанные метки для элементов выборки `X`

#### Параметры конструктора
`X, y` - обучающая выборка и соответствующие ей метки классов. Из нее нужно получить выборку для построения дерева при помощи bagging. Out-of-bag выборку нужно запомнить, она понадобится потом.

`criterion="gini"` - задает критерий, который будет использоваться при построении дерева. Возможные значения: `"gini"`, `"entropy"`.

`max_depth=None` - ограничение глубины дерева. Если `None` - глубина не ограничена

`min_samples_leaf=1` - минимальное количество элементов в каждом листе дерева.

`max_features="auto"` - количество признаков, которые могут использоваться в узле. Если `"auto"` - равно `sqrt(X.shape[1])`

In [3]:
def gini(x: np.ndarray) -> float:
    """
    Считает коэффициент Джини для массива меток x.
    """
    labels = np.unique(x)
    res = 0
    for y in labels:
        p = np.sum(x == y) / x.shape[0]
        res += p * (1 - p)
    return res

def entropy(x: np.ndarray) -> float:
    """
    Считает энтропию для массива меток x.
    """
    labels = np.unique(x)
    res = 0
    for y in labels:
        p = np.sum(x == y) / x.shape[0]
        res -= p * np.log(p)
    return res

def gain(all_y: np.ndarray, left_y: np.ndarray, right_y: np.ndarray, criterion: str) -> float:
    """
    Считает информативность разбиения массива меток.

    Parameters
    ----------
    left_y : np.ndarray
        Левая часть разбиения.
    right_y : np.ndarray
        Правая часть разбиения.
    criterion : Callable
        Критерий разбиения.
    """
    if criterion == 'gini':
        foo = gini
    else:
        foo = entropy
    return all_y.shape[0] * foo(all_y) - (left_y.shape[0] * foo(left_y) + right_y.shape[0] * foo(right_y)) / (all_y).shape[0]

In [4]:
class DecisionTreeLeaf:
    """
    Attributes
    ----------
    y : Тип метки (напр., int или str)
        Метка класса, который встречается чаще всего среди элементов листа дерева
    """

    def __init__(self, labels):
        self.prob_distribution = {}
        uniq_labels = np.unique(labels)
        for x in uniq_labels:
            self.prob_distribution[x] = (np.sum(labels == x) / len(labels))
        self.y = uniq_labels[np.argmax(list(self.prob_distribution.values()))]


class DecisionTreeNode:
    """

    Attributes
    ----------
    split_dim : int
        Измерение, по которому разбиваем выборку.
    split_value : float
        Значение, по которому разбираем выборку.
    left : Union[DecisionTreeNode, DecisionTreeLeaf]
        Поддерево, отвечающее за случай x[split_dim] < split_value.
    right : Union[DecisionTreeNode, DecisionTreeLeaf]
        Поддерево, отвечающее за случай x[split_dim] >= split_value.
    """

    def __init__(self, split_dim: int, split_value: float, left, right):
        self.split_dim = split_dim
        self.split_value = split_value
        self.left = left
        self.right = right

In [63]:
class DecisionTree:
    def __init__(self, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto"):
        # self.X = X
        # self.y = y
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.out_of_bag_X = None
        self.out_of_bag_y = None


    def build_tree(self, X, y, cur_depth):
        if len(np.unique(y)) == 1 or X.shape[0] < 2 * self.min_samples_leaf:
            return DecisionTreeLeaf(y)
        if self.max_depth and cur_depth >= self.max_depth:
            return DecisionTreeLeaf(y)

        if self.max_features == 'auto':
            feature_numb_list = np.random.choice(X.shape[1], int(np.sqrt(X.shape[1])), replace=False)
        else:
            feature_numb_list = np.random.choice(X.shape[1], self.max_features, replace=False)

        new_left, new_right = None, None
        X_left, X_right = None, None
        split_feature, split_value = None, None
        max_gain = 0
        for feature in feature_numb_list:
            for uni_f in np.unique(X[:, feature]):
                left_y = y[X[:, feature] < uni_f]
                right_y = y[X[:, feature] >= uni_f]
                if len(left_y) < self.min_samples_leaf or len(right_y) < self.min_samples_leaf:
                    continue
                cur_gain = gain(y, left_y, right_y, criterion=self.criterion)
                if cur_gain > max_gain:
                    max_gain = cur_gain
                    new_left, new_right = left_y, right_y
                    X_left, X_right = X[X[:, feature] < uni_f], X[X[:, feature] >= uni_f]
                    split_feature = feature
                    split_value = uni_f

        if max_gain == 0:
            return DecisionTreeLeaf(y)
        left_node = self.build_tree(X_left, new_left, cur_depth + 1)
        right_node = self.build_tree(X_right, new_right, cur_depth + 1)
        return DecisionTreeNode(split_feature, split_value, left_node, right_node)


    def fit(self, X: np.ndarray, y: np.ndarray):
        """
        Строит дерево решений по обучающей выборке.

        Parameters
        ----------
        X : np.ndarray
            Обучающая выборка.
        y : np.ndarray
            Вектор меток классов.
        """
        self.root = self.build_tree(X, y, 0)

    def go(self, node, x):
        if isinstance(node, DecisionTreeLeaf):
            return node.prob_distribution
        if x[node.split_dim] < node.split_value:
            return self.go(node.left, x)
        return self.go(node.right, x)

    def predict(self, X):
        proba = []
        for x in X:
            proba.append(self.go(self.root, x))
        return [max(p.keys(), key=lambda k: p[k]) for p in proba]

Проверка

In [65]:
def synthetic_dataset(size):
    X = [(np.random.randint(0, 2), np.random.randint(0, 2), i % 6 == 3,
          i % 6 == 0, i % 3 == 2, np.random.randint(0, 2)) for i in range(size)]
    y = [i % 3 for i in range(size)]
    return np.array(X), np.array(y)

X, y = synthetic_dataset(1000)
new_tree = DecisionTree()
new_tree.fit(X, y)
np.mean(new_tree.predict(X) == y)

0.854

### Задание 2 (2 балла)
Теперь реализуем сам Random Forest. Идея очень простая: строим `n` деревьев, а затем берем модальное предсказание.

#### Параметры конструктора
`n_estimators` - количество используемых для предсказания деревьев.

Остальное - параметры деревьев.

#### Методы
`fit(X, y)` - строит `n_estimators` деревьев по выборке `X`.

`predict(X)` - для каждого элемента выборки `X` возвращает самый частый класс, который предсказывают для него деревья.

In [119]:
class RandomForestClassifier:
    def __init__(self, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto", n_estimators=10):
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.n_estimators = n_estimators
        self.trees = []
        self.unique = None
    
    def fit(self, X, y):
        self.unique = np.unique(y)
        for i in range(self.n_estimators):
            new_tree = DecisionTree(criterion=self.criterion,
                                    max_depth=self.max_depth,
                                    min_samples_leaf=self.min_samples_leaf,
                                    max_features=self.max_features)
            ind = np.random.choice(X.shape[0], X.shape[0])
            out_ind = list(set(np.arange(X.shape[0])) - set(np.unique(ind)))
            # print(out_ind)
            new_tree.out_of_bag_X = X[out_ind].copy()
            new_tree.out_of_bag_y = y[out_ind].copy()
            new_tree.fit(X[ind], y[ind])
            self.trees.append(new_tree)

    
    def predict(self, X):
        pred = np.array([tree.predict(X) for tree in self.trees])
        # print(pred)
        res = []
        for i in range(pred.shape[1]):
            count = []
            for x in self.unique:
                count.append(np.sum(pred[:, i] == x))
            res.append(self.unique[np.argmax(count)])
        return np.array(res)

Проверка

In [121]:
def synthetic_dataset(size):
    X = [(np.random.randint(0, 2), np.random.randint(0, 2), i % 6 == 3,
          i % 6 == 0, i % 3 == 2, np.random.randint(0, 2)) for i in range(size)]
    y = [i % 3 for i in range(size)]
    return np.array(X), np.array(y)

X, y = synthetic_dataset(1000)
rfc = RandomForestClassifier(n_estimators=3)
rfc.fit(X, y)
print("Accuracy:", np.mean(rfc.predict(X) == y))
# print("Importance:", feature_importance(rfc))

Accuracy: 0.919


### Задание 3 (2 балла)
Часто хочется понимать, насколько большую роль играет тот или иной признак для предсказания класса объекта. Есть различные способы посчитать его важность. Один из простых способов сделать это для Random Forest - посчитать out-of-bag ошибку предсказания `err_oob`, а затем перемешать значения признака `j` и посчитать ее (`err_oob_j`) еще раз. Оценкой важности признака `j` для одного дерева будет разность `err_oob_j - err_oob`, важность для всего леса считается как среднее значение важности по деревьям.

Реализуйте функцию `feature_importance`, которая принимает на вход Random Forest и возвращает массив, в котором содержится важность для каждого признака.

In [1]:
def feature_importance(rfc):
    res = []
    for tree in rfc.trees:
        X = tree.out_of_bag_X
        y = tree.out_of_bag_y
        err_oob = np.mean(tree.predict(X) == y)
        errors = []
        for j in range(X.shape[1]):
            new_X = X.copy()
            new_X[:, j] = new_X[:, j][np.random.choice(new_X.shape[0], new_X.shape[0], replace=False)]
            err_oob_j = np.mean(tree.predict(new_X) == y)
            errors.append(err_oob - err_oob_j)
        res.append(errors)
    res = np.mean(res, axis=0)
    return res


def most_important_features(importance, names, k=20):
    # Выводит названия k самых важных признаков
    idicies = np.argsort(importance)[::-1][:k]
    return np.array(names)[idicies]

Наконец, пришло время протестировать наше дерево на простом синтетическом наборе данных. В результате точность должна быть примерно равна `1.0`, наибольшее значение важности должно быть у признака с индексом `4`, признаки с индексами `2` и `3`  должны быть одинаково важны, а остальные признаки - не важны совсем.

In [123]:
def synthetic_dataset(size):
    X = [(np.random.randint(0, 2), np.random.randint(0, 2), i % 6 == 3, 
          i % 6 == 0, i % 3 == 2, np.random.randint(0, 2)) for i in range(size)]
    y = [i % 3 for i in range(size)]
    return np.array(X), np.array(y)

X, y = synthetic_dataset(1000)
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X, y)
print("Accuracy:", np.mean(rfc.predict(X) == y))
print("Importance:", feature_importance(rfc))

Accuracy: 1.0
Importance: [ 5.13712853e-04 -1.51694042e-04  1.56868444e-01  1.67775870e-01
  3.35448314e-01 -1.45176605e-05]


### Задание 4 (1 балл)
Теперь поработаем с реальными данными.

Выборка состоит из публичных анонимизированных данных пользователей социальной сети Вконтакте. Первые два столбца отражают возрастную группу (`zoomer`, `doomer` и `boomer`) и пол (`female`, `male`). Все остальные столбцы являются бинарными признаками, каждый из них определяет, подписан ли пользователь на определенную группу/публичную страницу или нет.\
\
Необходимо обучить два классификатора, один из которых определяет возрастную группу, а второй - пол.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются. Лес должен строиться за какое-то разумное время.

In [124]:
def read_dataset(path):
    dataframe = pandas.read_csv(path, header=0)
    dataset = dataframe.values.tolist()
    random.shuffle(dataset)
    y_age = [row[0] for row in dataset]
    y_sex = [row[1] for row in dataset]
    X = [row[2:] for row in dataset]
    
    return np.array(X), np.array(y_age), np.array(y_sex), list(dataframe.columns)[2:]

In [125]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)

#### Возраст

In [127]:
%%time
rfc = RandomForestClassifier(n_estimators=10)
rfc.fit(X_train, y_age_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i+1) + ".", name)

Accuracy: 0.7175283732660782
Most important features:
1. ovsyanochan
2. 4ch
3. mudakoff
4. rhymes
5. styd.pozor
6. dayvinchik
7. rapnewrap
8. pravdashowtop
9. tumblr_vacuum
10. reflexia_our_feelings
11. leprum
12. iwantyou
13. bot_maxim
14. pixel_stickers
15. fuck_humor
16. i_des
17. pozor
18. top_screens
19. bog_memes
20. xfilm
CPU times: user 1min 29s, sys: 76.6 ms, total: 1min 29s
Wall time: 1min 30s


#### Пол

In [128]:
rfc = RandomForestClassifier(n_estimators=10)
rfc.fit(X_train, y_sex_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i+1) + ".", name)

Accuracy: 0.8461538461538461
Most important features:
1. 40kg
2. modnailru
3. girlmeme
4. mudakoff
5. i_d_t
6. cook_good
7. 9o_6o_9o
8. reflexia_our_feelings
9. igm
10. thesmolny
11. sh.cook
12. femalemem
13. bon
14. be.beauty
15. be.women
16. rapnewrap
17. zerofat
18. woman.blog
19. recipes40kg
20. 4ch


### CatBoost
В качестве аьтернативы попробуем CatBoost. 

Устаниовить его можно просто с помощью `pip install catboost`. Туториалы можно найти, например, [здесь](https://catboost.ai/docs/concepts/python-usages-examples.html#multiclassification) и [здесь](https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb). Главное - не забудьте использовать `loss_function='MultiClass'`.\
\
Сначала протестируйте CatBoost на синтетических данных. Выведите точность и важность признаков.

In [131]:
X, y = synthetic_dataset(1000)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

In [133]:
rfc = CatBoostClassifier(
    custom_loss=['Accuracy'],
    random_seed=42,
    logging_level='Silent',
    loss_function='MultiClass'
)
rfc.fit(X_train, y_train)

In [135]:
X, y = synthetic_dataset(1000)
print("Accuracy:", np.mean(rfc.predict(X_test).flatten() == y_test))
print("Importance:", rfc.get_feature_importance())

Accuracy: 1.0
Importance: [1.45494257e-02 8.91216472e-03 2.92394951e+01 2.73205004e+01
 4.34095828e+01 6.96007876e-03]


### Задание 5 (3 балла)
Попробуем применить один из используемых на практике алгоритмов. В этом нам поможет CatBoost. Также, как и реализованный ними RandomForest, применим его для определения пола и возраста пользователей сети Вконтакте, выведите названия наиболее важных признаков так же, как в задании 3.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются.

In [139]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)
X_train, X_eval, y_age_train, y_age_eval, y_sex_train, y_sex_eval = train_test_split(X_train, y_age_train, y_sex_train, train_size=0.8)

#### Возраст

In [140]:
rfc = CatBoostClassifier(
    custom_loss=['Accuracy'],
    random_seed=42,
    logging_level='Silent',
    loss_function='MultiClass'
)
rfc.fit(X_train, y_age_train)

In [142]:
print("Accuracy:", np.mean(rfc.predict(X_test).flatten() == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(rfc.get_feature_importance(), features, 10)):
    print(str(i+1) + ".", name)

Accuracy: 0.7414880201765448
Most important features:
1. styd.pozor
2. ovsyanochan
3. 4ch
4. mudakoff
5. rhymes
6. dayvinchik
7. leprum
8. fuck_humor
9. rapnewrap
10. xfilm


#### Пол

In [143]:
rfc = CatBoostClassifier(
    custom_loss=['Accuracy'],
    random_seed=42,
    logging_level='Silent',
    loss_function='MultiClass'
)
rfc.fit(X_train, y_sex_train)

In [144]:
print("Accuracy:", np.mean(rfc.predict(X_test).flatten() == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(rfc.get_feature_importance(), features, 10)):
    print(str(i+1) + ".", name)

Accuracy: 0.9003783102143758
Most important features:
1. 40kg
2. mudakoff
3. modnailru
4. girlmeme
5. 9o_6o_9o
6. 4ch
7. team
8. rapnewrap
9. fuck_humor
10. academyofman
